# Дообучение классификация спутниковые снимки

## Загрузка библиотек

In [ ]:
from torchvision import transforms
from torchvision import models
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import utils
import torch

import numpy as np
import matplotlib.pyplot as plt


from tqdm import tqdm
import os
import time
import copy
import random
from PIL import Image


# Обязательная ячейка

In [ ]:
path_to_all_data = r'C:\Users\user\Desktop\learn models'

## Изменение фотографий

In [ ]:
# подгон размера изображений под нейронку

# from torchvision import transforms

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        )
    ]
)

## Загрузка модели

In [ ]:
# загрузка модели
# from torchvision import models
model = models.vgg11(pretrained=True).eval()

## Загрузка датасета

In [ ]:
# загрузка датасета
# from torchvision import datasets
root_train = r'sputnics photo\train'
root_test = r'sputnics photo\test'

dataset = {
    'train': datasets.ImageFolder(
        root=os.path.join(path_to_all_data, root_train),
        transform=transform
    ),
    'valid': datasets.ImageFolder(
        root=os.path.join(path_to_all_data, root_test),
        transform=transform
    ),
}

dataset['train'].class_to_idx

## Батчи

In [ ]:
# деление на батчи
# from torch.utils.data import DataLoader

batch_size = 5
num_workers = 3 # распараллеливание
dataloaders = {
    'train': DataLoader(
        dataset['train'], 
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    ),
    'valid': DataLoader(
        dataset['valid'], 
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    )
}

## Отображение батча

In [ ]:
# отображение батча
real_batch = next(iter(dataloaders['train']))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(utils.make_grid(real_batch[0][:64], padding=2, normalize=True),(1,2,0)));

При дообучении (файтюнинге) меняют только конец нейронки - полносвязные слои. Допустим в этой нейронке выход на 1000 классов. А мне нужно всего 4. Переобучать сверточные слои мне не нужно - они и так выделяют то что нужно, а вот слои которые классифицируют мои картинки уже нужно менять.

## Заморозка слоев

In [ ]:
# заморозка обучения модели, перевод на GPU
device = 'cpu'

model = models.vgg11(pretrained=True, progress=False).to(device)

In [ ]:
# замараживаем параметры  сверточных слоев
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# замена слоя отвечающего за классификацию
model.classifier[6] = torch.nn.Linear(4096, 4).to(device)


В прошлой подсказке наебал. Тут мы меняем только последний слой отвечающий за классификацию.

## Оптимизатор, критерий оптимизации

In [ ]:
params_to_update = model.parameters()

params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

optimizer = torch.optim.Adam(params_to_update, lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

## Функция обучения

In [ ]:
# функция для обучения модели

def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'valid':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

## Обучение

In [ ]:
model, val_acc_history = train_model(model, dataloaders, criterion, optimizer, num_epochs=1)

## Сохранение модели

In [ ]:
std_info = 'models.vgg11(pretrained=True, progress=False)'
checkpoint = {
            'info': std_info,
            'state_model': model.state_dict(),
        }
torch.save(checkpoint, os.path.join(path_to_all_data, 'sputnics photo/model_fine_tune_classifier_epoche_1.pt'))

## Загрузка модели

In [ ]:
load_model_state = torch.load(os.path.join(path_to_all_data, 'sputnics photo/model_fine_tune_classifier_epoche_1.pt'))

In [ ]:
load_model = models.vgg11(pretrained=True, progress=False).to(device)
load_model.classifier[6] = torch.nn.Linear(4096, 4).to(device)

In [ ]:
load_model.load_state_dict(load_model_state['state_model'])
load_model.eval()

In [ ]:
real_batch = next(iter(dataloaders['train']))
print(real_batch)
print(type(real_batch))

In [ ]:
real_batch = next(iter(dataloaders['train']))

In [ ]:
real_batch[1]

In [ ]:
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(utils.make_grid(real_batch[0][:64], padding=2, normalize=True),(1,2,0)));

In [ ]:
real_batch[0][0].shape

In [ ]:
load_model(real_batch[0][1][np.newaxis])